<img src="images/splash3.png">

### Setup

In [1]:
import sqlite3 as lite
import sys
import coinmarketcap
import requests
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_rows", 200)

### Fetch data
Uses cryptocompare API to get historical data for target coin the last 3 months, coinmarketcap api to get top-100 coin names.

In [2]:
def fetch_top100():
    url = 'https://api.coinmarketcap.com/v2/ticker/'
    page = requests.get(url)
    data = page.json()['data']
    coinlist = pd.DataFrame(data)
    return coinlist

In [3]:
def fetch_coindata(symbol, comparison_symbol, limit, aggregate):    
    url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)       
    page = requests.get(url)
    print(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['mid'] = df[["high", "low"]].mean(axis=1)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d).strftime('%d-%m-%Y %H:%M:%S') for d in df.time]
    #df = df.set_index('timestamp')        
    return df

### Store values to database

In [4]:
def write_to_db():
    #Connect to DB
    con = None
    con = lite.connect('crypto.db')

    #Dump to SQL
    with con:
        market_data.to_sql(name=top_coins[symbol], con=con, flavor='sqlite', schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

    """"
    #Read SQL    
    with con:        
        cur = con.cursor()    
        cur.execute("SELECT * FROM btc")

        rows = cur.fetchall()

        for row in rows:
            print(row)  
    """

In [5]:
top_coins = fetch_top100().loc['symbol']

# market_data = fetch_coindata(top_coins[1],'BTC','2000','12')
# write_to_db()

for symbol in range(1, 100):
    try:
        market_data = fetch_coindata(top_coins[symbol],'BTC','2000','12')
        write_to_db()
        print("Wrote to: ", top_coins[symbol])
    except:
        print("Error noticed, skipping coin ", top_coins[symbol])

https://min-api.cryptocompare.com/data/histohour?fsym=ETH&tsym=BTC&limit=2000&aggregate=12


/usr/local/lib/python3.5/dist-packages/pandas/io/sql.py:531: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


Wrote to:  ETH
https://min-api.cryptocompare.com/data/histohour?fsym=SC&tsym=BTC&limit=2000&aggregate=12
Wrote to:  SC
https://min-api.cryptocompare.com/data/histohour?fsym=DGB&tsym=BTC&limit=2000&aggregate=12
Wrote to:  DGB
https://min-api.cryptocompare.com/data/histohour?fsym=REP&tsym=BTC&limit=2000&aggregate=12
Wrote to:  REP
https://min-api.cryptocompare.com/data/histohour?fsym=DCR&tsym=BTC&limit=2000&aggregate=12
Wrote to:  DCR
https://min-api.cryptocompare.com/data/histohour?fsym=PIVX&tsym=BTC&limit=2000&aggregate=12
Wrote to:  PIVX
https://min-api.cryptocompare.com/data/histohour?fsym=RDD&tsym=BTC&limit=2000&aggregate=12
Wrote to:  RDD
https://min-api.cryptocompare.com/data/histohour?fsym=LSK&tsym=BTC&limit=2000&aggregate=12
Wrote to:  LSK
https://min-api.cryptocompare.com/data/histohour?fsym=DGD&tsym=BTC&limit=2000&aggregate=12
Wrote to:  DGD
https://min-api.cryptocompare.com/data/histohour?fsym=STEEM&tsym=BTC&limit=2000&aggregate=12
Wrote to:  STEEM
https://min-api.cryptocompa

https://min-api.cryptocompare.com/data/histohour?fsym=ELA&tsym=BTC&limit=2000&aggregate=12
Wrote to:  ELA
https://min-api.cryptocompare.com/data/histohour?fsym=POLY&tsym=BTC&limit=2000&aggregate=12
Wrote to:  POLY
https://min-api.cryptocompare.com/data/histohour?fsym=HT&tsym=BTC&limit=2000&aggregate=12
Wrote to:  HT
https://min-api.cryptocompare.com/data/histohour?fsym=FSN&tsym=BTC&limit=2000&aggregate=12
Wrote to:  FSN
https://min-api.cryptocompare.com/data/histohour?fsym=ONT&tsym=BTC&limit=2000&aggregate=12
Wrote to:  ONT
https://min-api.cryptocompare.com/data/histohour?fsym=BTCP&tsym=BTC&limit=2000&aggregate=12
Wrote to:  BTCP
https://min-api.cryptocompare.com/data/histohour?fsym=CENNZ&tsym=BTC&limit=2000&aggregate=12
Error noticed, skipping coin  CENNZ
https://min-api.cryptocompare.com/data/histohour?fsym=LOOM&tsym=BTC&limit=2000&aggregate=12
Wrote to:  LOOM
https://min-api.cryptocompare.com/data/histohour?fsym=WAN&tsym=BTC&limit=2000&aggregate=12
Wrote to:  WAN
https://min-api.cry